<a href="https://colab.research.google.com/github/Sallb679/SafeGraph-Analysis/blob/main/Econ433_proj3_Boubacar_Sall_Cracker_Barrel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tabulate import tabulate

The goal of this project is to help you learn data merge, group comparison, and basic regression analysis.



The project will use two datasets:

The “long sample” you have created in Project 1, which should already have each observation defined as a place-day, and include variables such as date, dailyvisits, dayofweek, and some geographic variables in each observation. By definition, the data focuses on your restaurant chain.
A comparison "long sample" that you would generate by replicating the data you created in Project 1 but using another restaurant chain as a benchmark (more details below).
When you turn in your jupyter notebook on ELMS (in both .ipynb and .pdf), please label clearly your name and the restaurant chain you are focusing on. For example, if your name is Joe Smith and you focus on Subway, you should name your project 3 submissions as Econ433-proj3-Joe-Smith-Subway.ipynb and Econ433-proj3-Joe-Smith-Subway.pdf.

Within your jupyter notebook, please use markdown cells to describe which python commands and outcome are for which question in the project. Make sure your notebook is reader-friendly, includes all key information, and is free of unnecessary text (such as the error message before you fix the problem).



Part 1: Define your benchmark comparison



Question 1a (1 point): Choose another restaurant chain in the most original nationwide data as the benchmark for your own restaurant chain. This benchmark chain could be a major competitor of your own chain, a chain that your chain aspires to grow into, or a chain that your chain may want to acquire for future business. Whatever your choice is, please:

explain why you choose this benchmark chain, and
what you would like to learn most by comparing your own chain with this benchmark chain.


In [ ]:
# Define the name of the restaurant chain
restaurant_chain_name = "Cracker Barrel"

# Initialize an empty list to store the filtered Data
filtered_data = []

# Loop through the years from 2018 to 2022
for year in range(2018, 2023):
    # Generate the file path for the current year using an f-string
    file_path = f'/content/drive/MyDrive/ECON_433_Project_1_data/weekly_patterns_{year}_sample.csv.zip'

    # Load the dataset
    data = pd.read_csv(file_path, compression='zip')

    # Filter rows where the restaurant chain matches
    filtered_df = data[data['brands'] == restaurant_chain_name]

    # Append the filtered DataFrame to the list
    filtered_data.append(filtered_df)

# Concatenate all the filtered DataFrames into one DataFrame
concatenated_data = pd.concat(filtered_data)

# Print the total number of observations
total_observations = len(concatenated_data)
print(f"Total number of observations for {restaurant_chain_name}: {total_observations}")

Total number of observations for Cracker Barrel: 28722


I chose Cracker Barrel as the benchmark chain for comparison with Chili's Grill and Bar because it is considered Chili's biggest competitor in the casual dining market. By comparing these two chains, I aim to understand how external factors, such as state characteristics and national events, impact their relative performance in terms of customer visits.

Question 1b (1 point): once you choose your benchmark chain, repeat the data generating process of Project 1 for this benchmark chain. The resulting long sample -- let us call it "Benchmark Long" -- should have exactly the same data structure as the long sample you have created for your own chain at the end of Project 1. For ease of illustration, I will refer to the long sample of your own chain as "Focal Long".




In [ ]:
# Check data types of each column in the concatenated DataFrame
print(concatenated_data.dtypes)


# Count unique places using 'placekey'
unique_places = concatenated_data['placekey'].nunique()
print(f"Number of unique places: {unique_places}")

# Save the wide sample to a CSV file
concatenated_data.to_csv('/content/drive/MyDrive/ECON_433_Project_1_data/chilis_wide_sample.csv', index=False)

placekey               object
city                   object
region                 object
date_range_start       object
date_range_end         object
raw_visit_counts        int64
visits_by_day          object
safegraph_brand_ids    object
naics_code              int64
postal_code             int64
brands                 object
dtype: object
Number of unique places: 125


In [ ]:
# Remove timezone information directly from 'date_range_start' and 'date_range_end' columns
concatenated_data['date_range_start'] = concatenated_data['date_range_start'].str.replace(r'([+-]\d{2}:\d{2})', '', regex=True)
concatenated_data['date_range_end'] = concatenated_data['date_range_end'].str.replace(r'([+-]\d{2}:\d{2})', '', regex=True)

# Convert the cleaned date strings back to datetime format
concatenated_data['date_range_start'] = pd.to_datetime(concatenated_data['date_range_start'], errors='coerce')
concatenated_data['date_range_end'] = pd.to_datetime(concatenated_data['date_range_end'], errors='coerce')

# Check for invalid dates
invalid_dates_start = concatenated_data['date_range_start'].isna().sum()
invalid_dates_end = concatenated_data['date_range_end'].isna().sum()

# Print out how many invalid dates were found
print(f"Invalid 'date_range_start' entries: {invalid_dates_start}")
print(f"Invalid 'date_range_end' entries: {invalid_dates_end}")

# Find the earliest and latest valid dates
beginning_date = concatenated_data['date_range_start'].min()
ending_date = concatenated_data['date_range_end'].max()

# Print the results
print(f"Beginning date in the wide sample: {beginning_date}")
print(f"Ending date in the wide sample: {ending_date}")



Invalid 'date_range_start' entries: 0
Invalid 'date_range_end' entries: 0
Beginning date in the wide sample: 2018-01-01 00:00:00
Ending date in the wide sample: 2023-01-02 00:00:00


In [ ]:
concatenated_data.head(5)


,placekey,city,region,date_range_start,date_range_end,raw_visit_counts,visits_by_day,safegraph_brand_ids,naics_code,postal_code,brands
301127,222-222@8fy-d78-ysq,Kissimmee,FL,2018-09-03,2018-09-10,292,"[69,26,23,22,36,52,64]",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,34746,Cracker Barrel
301128,222-222@5pj-89c-pn5,Calvert City,KY,2018-07-16,2018-07-23,307,"[24,24,42,41,48,49,79]",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,42029,Cracker Barrel
301129,222-222@63p-vj5-9j9,State College,PA,2018-06-25,2018-07-02,79,"[4,10,9,9,8,17,22]",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,16803,Cracker Barrel
301130,222-222@63p-vj5-9j9,State College,PA,2018-01-29,2018-02-05,66,"[9,5,7,9,13,15,8]",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,16803,Cracker Barrel
301131,222-222@629-mnm-vcq,Fishkill,NY,2018-08-20,2018-08-27,323,"[40,36,35,35,49,68,60]",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,12524,Cracker Barrel


In [ ]:
#print the data types
print(concatenated_data.dtypes)


placekey                       object
city                           object
region                         object
date_range_start       datetime64[ns]
date_range_end         datetime64[ns]
raw_visit_counts                int64
visits_by_day                  object
safegraph_brand_ids            object
naics_code                      int64
postal_code                     int64
brands                         object
dtype: object


In [ ]:
# Remove brackets and split the string into a list of integers
concatenated_data['visits_by_day'] = concatenated_data['visits_by_day'].str.strip("[]")  # Remove brackets
daily_visits = concatenated_data['visits_by_day'].str.split(',', expand=True).astype(int)  # Split and convert to int

# Assign to new columns
concatenated_data[['dailyvisits1', 'dailyvisits2', 'dailyvisits3', 'dailyvisits4', 'dailyvisits5', 'dailyvisits6', 'dailyvisits7']] = daily_visits

# Check the number of columns in the wide sample
num_columns = concatenated_data.shape[1]
print(f"Total number of variables in the wide sample now: {num_columns}")

Total number of variables in the wide sample now: 18


In [ ]:
# Step 1: Reshape the DataFrame from wide to long format
long_sample = pd.melt(concatenated_data,
                      id_vars=['placekey', 'city', 'region', 'date_range_start', 'date_range_end',
                                'raw_visit_counts', 'visits_by_day', 'safegraph_brand_ids',
                                'naics_code', 'postal_code', 'brands'],
                      value_vars=['dailyvisits1', 'dailyvisits2', 'dailyvisits3',
                                  'dailyvisits4', 'dailyvisits5', 'dailyvisits6', 'dailyvisits7'],
                      var_name='day',
                      value_name='dailyvisits')

# Step 2: Convert the 'dailyvisits' column to integer
long_sample['dailyvisits'] = long_sample['dailyvisits'].astype(int)

# Step 3: Final count of observations after resolving issues
num_observations_final = long_sample.shape[0]
print(f"Number of observations: {num_observations_final}")

Number of observations: 201054


In [ ]:
# Step 1: Extract the year from the start date
long_sample['year'] = long_sample['date_range_start'].dt.year

# Step 2: Aggregate data by year
yearly_summary = long_sample.groupby('year').agg(
    start_date=('date_range_start', 'min'),
    end_date=('date_range_end', 'max'),
    num_observations=('dailyvisits', 'size'),
    mean_dailyvisits=('dailyvisits', 'mean')
).reset_index()

# Convert the mean_dailyvisits to two decimal places and format the dates
yearly_summary['mean_dailyvisits'] = yearly_summary['mean_dailyvisits'].round(2)
yearly_summary['start_date'] = yearly_summary['start_date'].dt.strftime('%Y-%m-%d')
yearly_summary['end_date'] = yearly_summary['end_date'].dt.strftime('%Y-%m-%d')

# Step 4: Display the summary table using tabulate
print(tabulate(yearly_summary, headers='keys', tablefmt='grid'))

+----+--------+--------------+------------+--------------------+--------------------+
|    |   year | start_date   | end_date   |   num_observations |   mean_dailyvisits |
+====+========+==============+============+====================+====================+
|  0 |   2018 | 2018-01-01   | 2019-01-07 |              32578 |              37.18 |
+----+--------+--------------+------------+--------------------+--------------------+
|  1 |   2019 | 2019-01-07   | 2020-01-06 |              32284 |              50.08 |
+----+--------+--------------+------------+--------------------+--------------------+
|  2 |   2020 | 2020-01-06   | 2021-01-04 |              45465 |              37.47 |
+----+--------+--------------+------------+--------------------+--------------------+
|  3 |   2021 | 2021-01-04   | 2022-01-03 |              45500 |              48.37 |
+----+--------+--------------+------------+--------------------+--------------------+
|  4 |   2022 | 2022-01-03   | 2023-01-02 |           

In [ ]:
# Step 1: Create a date variable based on the start date and day number
long_sample['date'] = long_sample['date_range_start'] + pd.to_timedelta(
    long_sample['day'].str.extract('(\d+)')[0].astype(int) - 1, unit='d'
)

# Step 2: Define the dayofweek variable (1=Monday, ..., 7=Sunday)
long_sample['dayofweek'] = long_sample['date'].dt.dayofweek + 1

# Step 3: Aggregate dailyvisits by dayofweek
dayofweek_summary = long_sample.groupby('dayofweek').agg(
    num_observations=('dailyvisits', 'size'),
    mean_dailyvisits=('dailyvisits', 'mean')
).reset_index()

# Map numeric dayofweek to actual day names
day_names = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
dayofweek_summary['dayofweek_name'] = dayofweek_summary['dayofweek'].map(day_names)

# Format mean_dailyvisits to two decimal places
dayofweek_summary['mean_dailyvisits'] = dayofweek_summary['mean_dailyvisits'].round(2)

# Step 4: Determine which day has the highest average daily visits
highest_avg_day = dayofweek_summary.loc[dayofweek_summary['mean_dailyvisits'].idxmax()]

# Step 5: Display the summary table using tabulate
print(tabulate(dayofweek_summary[['dayofweek_name', 'num_observations', 'mean_dailyvisits']],
               headers=['Day of Week', 'Num Observations', 'Mean Daily Visits'],
               tablefmt='grid'))

# Step 6: Display the highest average daily visits
print(f"\nThe day with the highest average daily visits is {highest_avg_day['dayofweek_name']} "
      f"with an average of {highest_avg_day['mean_dailyvisits']:.2f} visits.")

+----+---------------+--------------------+---------------------+
|    | Day of Week   |   Num Observations |   Mean Daily Visits |
+====+===============+====================+=====================+
|  0 | Monday        |              28722 |               34.15 |
+----+---------------+--------------------+---------------------+
|  1 | Tuesday       |              28722 |               31.62 |
+----+---------------+--------------------+---------------------+
|  2 | Wednesday     |              28722 |               34.33 |
+----+---------------+--------------------+---------------------+
|  3 | Thursday      |              28722 |               38.02 |
+----+---------------+--------------------+---------------------+
|  4 | Friday        |              28722 |               49.33 |
+----+---------------+--------------------+---------------------+
|  5 | Saturday      |              28722 |               60.77 |
+----+---------------+--------------------+---------------------+
|  6 | Sun

In [ ]:
# Step 1: Calculate mean and standard deviation for weekdays (Monday to Thursday) and weekends (Friday to Sunday)
weekday_data = long_sample[long_sample['dayofweek'] <= 4]['dailyvisits']
weekend_data = long_sample[long_sample['dayofweek'] >= 5]['dailyvisits']

# Mean and standard deviation for weekdays
weekday_mean = weekday_data.mean()
weekday_std = weekday_data.std()

# Mean and standard deviation for weekends
weekend_mean = weekend_data.mean()
weekend_std = weekend_data.std()

# Step 2: Define thresholds as mean + 1 standard deviation
weekday_threshold = weekday_mean + weekday_std
weekend_threshold = weekend_mean + weekend_std

# Step 3: Create the manyvisits variable using the data-driven thresholds
long_sample['manyvisits'] = long_sample.apply(
    lambda row: 1 if (row['dailyvisits'] > weekend_threshold if row['dayofweek'] >= 5 else row['dailyvisits'] > weekday_threshold) else 0,
    axis=1
)

# Step 4: Choose a geographic variable (e.g., region)
geographic_var = 'region'

# Step 5: Tabulate the number of observations for each combination of manyvisits and the geographic variable
tabulation = long_sample.groupby([geographic_var, 'manyvisits']).size().unstack(fill_value=0)

# Add a column for total observations in each geographic unit
tabulation['total_observations'] = tabulation.sum(axis=1)

# Add a column for the percentage of manyvisits=1 in each geographic unit, rounded to 2 decimal places
tabulation['percent_manyvisits_1'] = ((tabulation[1] / tabulation['total_observations']) * 100).round(2)

# Step 6: Find the geographic unit with the highest number of manyvisits=1
highest_manyvisits_1 = tabulation[1].idxmax()
highest_manyvisits_1_count = tabulation[1].max()

# Step 7: Find the geographic unit with the highest percentage of manyvisits=1
highest_percent_manyvisits_1 = tabulation['percent_manyvisits_1'].idxmax()
highest_percent_manyvisits_1_value = tabulation['percent_manyvisits_1'].max()

# Step 8: Display the tabulation and results using tabulate
tabulated_data = tabulation.reset_index()
print(tabulate(tabulated_data, headers='keys', tablefmt='grid', showindex=False))

# Step 9: Display the highest number and percentage of manyvisits=1
print(f"\nThe geographic unit with the highest # of observations with manyvisits=1 is {highest_manyvisits_1} with {highest_manyvisits_1_count} observations.")
print(f"The geographic unit with the highest percentage of observations with manyvisits=1 is {highest_percent_manyvisits_1} with {highest_percent_manyvisits_1_value:.2f}% observations.")

+----------+-------+------+----------------------+------------------------+
| region   |     0 |    1 |   total_observations |   percent_manyvisits_1 |
+==========+=======+======+======================+========================+
| AL       |  5755 | 3751 |                 9506 |                  39.46 |
+----------+-------+------+----------------------+------------------------+
| AR       |  2110 |  823 |                 2933 |                  28.06 |
+----------+-------+------+----------------------+------------------------+
| AZ       |  4686 |   67 |                 4753 |                   1.41 |
+----------+-------+------+----------------------+------------------------+
| CA       |  3503 |  144 |                 3647 |                   3.95 |
+----------+-------+------+----------------------+------------------------+
| CO       |  1747 |   80 |                 1827 |                   4.38 |
+----------+-------+------+----------------------+------------------------+
| FL       |

In [ ]:
# Step 1: Calculate the threshold for core business areas (e.g., 75th percentile of observations by postal_code)
obs_per_postal_code = long_sample.groupby('postal_code')['dailyvisits'].size()
threshold = np.percentile(obs_per_postal_code, 75)

# Step 2: Define the core_biz_area variable (1 if postal_code has observations above the threshold, 0 otherwise)
long_sample['core_biz_area'] = long_sample['postal_code'].apply(
    lambda x: 1 if obs_per_postal_code[x] > threshold else 0
)

# Step 3: Aggregate data by core_biz_area
core_biz_summary = long_sample.groupby('core_biz_area').agg(
    num_observations=('dailyvisits', 'size'),
    mean_dailyvisits=('dailyvisits', 'mean')
).reset_index()

# Step 4: Display the aggregated data
print(tabulate(core_biz_summary, headers='keys', tablefmt='grid', showindex=False))

# Step 5: Interpret results (example reasoning)
core_area_visits = core_biz_summary[core_biz_summary['core_biz_area'] == 1]['mean_dailyvisits'].values[0]
non_core_area_visits = core_biz_summary[core_biz_summary['core_biz_area'] == 0]['mean_dailyvisits'].values[0]

print(f"Core areas have an average of {core_area_visits:.2f} daily visits compared to {non_core_area_visits:.2f} in non-core areas.")

+-----------------+--------------------+--------------------+
|   core_biz_area |   num_observations |   mean_dailyvisits |
+=================+====================+====================+
|               0 |             180880 |            44.8835 |
+-----------------+--------------------+--------------------+
|               1 |              20174 |            38.3198 |
+-----------------+--------------------+--------------------+
Core areas have an average of 38.32 daily visits compared to 44.88 in non-core areas.


In [ ]:
Benchmark_long = long_sample
Benchmark_long.head(5)

,placekey,city,region,date_range_start,date_range_end,raw_visit_counts,visits_by_day,safegraph_brand_ids,naics_code,postal_code,brands,day,dailyvisits,year,date,dayofweek,manyvisits,core_biz_area
0,222-222@8fy-d78-ysq,Kissimmee,FL,2018-09-03,2018-09-10,292,"69,26,23,22,36,52,64",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,34746,Cracker Barrel,dailyvisits1,69,2018,2018-09-03,1,1,0
1,222-222@5pj-89c-pn5,Calvert City,KY,2018-07-16,2018-07-23,307,"24,24,42,41,48,49,79",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,42029,Cracker Barrel,dailyvisits1,24,2018,2018-07-16,1,0,1
2,222-222@63p-vj5-9j9,State College,PA,2018-06-25,2018-07-02,79,"4,10,9,9,8,17,22",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,16803,Cracker Barrel,dailyvisits1,4,2018,2018-06-25,1,0,0
3,222-222@63p-vj5-9j9,State College,PA,2018-01-29,2018-02-05,66,"9,5,7,9,13,15,8",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,16803,Cracker Barrel,dailyvisits1,9,2018,2018-01-29,1,0,0
4,222-222@629-mnm-vcq,Fishkill,NY,2018-08-20,2018-08-27,323,"40,36,35,35,49,68,60",SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,12524,Cracker Barrel,dailyvisits1,40,2018,2018-08-20,1,0,0


I loaded my Focal long set below from my file directory i also repeated the same steps for my benchmark data above

In [ ]:
# Step 1: Load the CSV file into a DataFrame named 'Focal_long'
file_path = '/content/drive/MyDrive/ECON_433_Project_2_data/long_sample.csv'
Focal_long = pd.read_csv(file_path)


In [ ]:
# Convert the date range start and end columns to datetime
Focal_long['date_range_start'] = pd.to_datetime(Focal_long['date_range_start'])
Focal_long['date_range_end'] = pd.to_datetime(Focal_long['date_range_end'])

# Convert the 'date' column to datetime
Focal_long['date'] = pd.to_datetime(Focal_long['date'])

#Check the data types to confirm conversion
print(Focal_long.dtypes)


placekey                       object
city                           object
region                         object
date_range_start       datetime64[ns]
date_range_end         datetime64[ns]
raw_visit_counts                int64
visits_by_day                  object
safegraph_brand_ids            object
naics_code                      int64
postal_code                     int64
brands                         object
day                            object
dailyvisits                     int64
year                            int64
date                   datetime64[ns]
dayofweek                       int64
manyvisits                      int64
core_biz_area                   int64
dtype: object


In [ ]:
Focal_long.head(5)

,placekey,city,region,date_range_start,date_range_end,raw_visit_counts,visits_by_day,safegraph_brand_ids,naics_code,postal_code,brands,day,dailyvisits,year,date,dayofweek,manyvisits,core_biz_area
0,222-222@5pr-4pg-9fz,Lees Summit,MO,2018-12-24,2018-12-31,261,"26,1,43,45,52,53,41",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,64081,Chili's Grill & Bar,dailyvisits1,26,2018,2018-12-24,1,0,0
1,223-222@8f2-sqt-kj9,Mount Pleasant,TX,2018-08-27,2018-09-03,138,"10,24,14,13,23,24,30",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,75455,Chili's Grill & Bar,dailyvisits1,10,2018,2018-08-27,1,0,0
2,223-222@5pv-mqc-t5f,Peoria,IL,2018-03-19,2018-03-26,111,"10,10,16,15,21,16,23",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,61614,Chili's Grill & Bar,dailyvisits1,10,2018,2018-03-19,1,0,0
3,zzw-222@62j-wsq-ysq,Dover,NH,2018-05-07,2018-05-14,113,"13,15,18,9,26,19,13",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,3820,Chili's Grill & Bar,dailyvisits1,13,2018,2018-05-07,1,0,0
4,228-222@5qv-x6c-gzf,Euless,TX,2018-02-12,2018-02-19,175,"26,14,34,9,34,34,24",SG_BRAND_072efa4069c7217d82229845c60e6a0d,722511,76039,Chili's Grill & Bar,dailyvisits1,26,2018,2018-02-12,1,0,0


Questions to answer:

What is the unit of observation in each long sample?
How many observations do you have in “Benchmark Long" and "Focal Long" respectively?
Question 1c (1 point): Do some simple comparison between the two long samples for sanity check:

How many unique chain units do you have in “Benchmark Long" and "Focal Long" respectively?
In how many unique states do your own chain and benchmark chain operate respectively?
Are the answers to the above two questions consistent with your expectation?

In [ ]:
#observations in Benchmark_long and Focal_long
print(f"Observations in Benchmark_long: {len(Benchmark_long)}")
print(f"Observations in Focal_long: {len(Focal_long)}")

# Compare summary statistics for daily visits
benchmark_stats = Benchmark_long['dailyvisits'].describe()
focal_stats = Focal_long['dailyvisits'].describe()

print("Benchmark Long Summary Statistics:")
print(benchmark_stats)

print("\nFocal Long Summary Statistics:")
print(focal_stats)

# Compare number of unique geographic units (e.g., region)
benchmark_unique_geo = Benchmark_long['region'].nunique()
focal_unique_geo = Focal_long['region'].nunique()

print(f"\nUnique regions in Benchmark Long: {benchmark_unique_geo}")
print(f"Unique regions in Focal Long: {focal_unique_geo}")



Observations in Benchmark_long: 201054
Observations in Focal_long: 393694
Benchmark Long Summary Statistics:
count    201054.000000
mean         44.224920
std          25.923503
min           0.000000
25%          26.000000
50%          41.000000
75%          59.000000
max         278.000000
Name: dailyvisits, dtype: float64

Focal Long Summary Statistics:
count    393694.000000
mean         26.854059
std          29.779666
min           0.000000
25%          13.000000
50%          22.000000
75%          34.000000
max         819.000000
Name: dailyvisits, dtype: float64

Unique regions in Benchmark Long: 30
Unique regions in Focal Long: 42


The unit of observation is a specific resturant locatation for a particular day from 2018-2023. Benchmark-Long has 201054 observations and Focal_long has 393694 observations. They operate in 30 and 42 states respectively and these numbers are consistent with my expectation.

Part 2: Merge data for comparison

Question 2a (2 points): To compare the two chains in each state-day, collapse each long sample into a new dataset by "state-day". We will call these two new datasets "BenchmarkSummary" and "FocalSummary".

Each of the two summary files should contain the following variables for the relevant chain:

the number of unique chain units in a state-day (name the variable Nunits);
total dailyvisits observed across all units of the chain in a state-day (name the variable SumDailyVisits),
maximum, minimum, and median dailyvisits per unit in a state-day (name them as MaxDailyVisitsPerUnit, MinDailyVisitsPerUnit, and MedDailyVisitsPerUnit).
Questions to answer:

What is the unit of observation in these two summary datasets?
How many observations are there in BenchmarkSummary?
How many observations are there in FocalSummary?
Why do you think the number of observations are similar (or different)? Are they consistent with your expectation?


In [ ]:
# Add 'dayofweek' to both Benchmark_long and Focal_long DataFrames
Benchmark_long['dayofweek'] = Benchmark_long['date'].dt.dayofweek
Focal_long['dayofweek'] = Focal_long['date'].dt.dayofweek

# Group by region-day for Benchmark_long
BenchmarkSummary = Benchmark_long.groupby(['region', 'date', 'dayofweek']).agg(
    Nunits=('placekey', 'nunique'),  # Count unique units per region-day using placekey
    SumDailyVisits=('dailyvisits', 'sum'),  # Total daily visits per region-day
    MaxDailyVisitsPerUnit=('dailyvisits', 'max'),  # Maximum daily visits per unit
    MinDailyVisitsPerUnit=('dailyvisits', 'min'),  # Minimum daily visits per unit
    MedDailyVisitsPerUnit=('dailyvisits', 'median')  # Median daily visits per unit
).reset_index()

# Group by region-day for Focal_long
FocalSummary = Focal_long.groupby(['region', 'date', 'dayofweek']).agg(
    Nunits=('placekey', 'nunique'),  # Count unique units per region-day using placekey
    SumDailyVisits=('dailyvisits', 'sum'),  # Total daily visits per region-day
    MaxDailyVisitsPerUnit=('dailyvisits', 'max'),  # Maximum daily visits per unit
    MinDailyVisitsPerUnit=('dailyvisits', 'min'),  # Minimum daily visits per unit
    MedDailyVisitsPerUnit=('dailyvisits', 'median')  # Median daily visits per unit
).reset_index()

# Number of observations
benchmark_obs = BenchmarkSummary.shape[0]
focal_obs = FocalSummary.shape[0]

print(f"Number of observations in BenchmarkSummary: {benchmark_obs}")
print(f"Number of observations in FocalSummary: {focal_obs}")



Number of observations in BenchmarkSummary: 54747
Number of observations in FocalSummary: 74788


The unit of observation in both BenchmarkSummary and FocalSummary is State-day. This means that each row in the dataset represents aggregated data for a specific state on a particular day, summarizing visits and chain unit activity in that state.
Number of observations in BenchmarkSummary: 54747
Number of observations in FocalSummary: 74788
The number of observations are different because
Focal operates in 42 states, while Benchmark operates in 30. With more regions, Focal has more chances to generate state-day observations. For instance, over 100 days, Focal could have 4,200 state-day observations (42 * 100), compared to Benchmark's 3,000 (30 * 100).
They are consistent with my expectations.

Question 2b (2 points): Merge BenchmarkSummary and FocalSummary by state-day.

Questions to answer:

How many observations can be matched between the two datasets?
How many observations appear in BenchmarkSummary but cannot be matched with FocalSummary?
How many observations appear in FocalSummary but cannot be matched with BenchmarkSummary?
Why do you observe imperfect matches? Provide explanations for both the matched parts and the unmatched parts.
From now on, only keep the observations that match between BenchmarkSummary and FocalSummary. Let us call this new data MergedSummary.



In [ ]:
# Merge BenchmarkSummary and FocalSummary by 'region' and 'date'
MergedSummary = pd.merge(BenchmarkSummary, FocalSummary, on=['region', 'date'], how='inner')

# Reset the index to make sure 'region' and 'date' are columns
MergedSummary.reset_index(drop=True, inplace=True)

# Step 2: Count matched and unmatched observations
matched_observations = MergedSummary.shape[0]

# Observations in BenchmarkSummary not matched with FocalSummary
unmatched_benchmark = BenchmarkSummary.merge(FocalSummary, on=['region', 'date'], how='left', indicator=True)
benchmark_unmatched_count = unmatched_benchmark[unmatched_benchmark['_merge'] == 'left_only'].shape[0]

# Observations in FocalSummary not matched with BenchmarkSummary
unmatched_focal = FocalSummary.merge(BenchmarkSummary, on=['region', 'date'], how='left', indicator=True)
focal_unmatched_count = unmatched_focal[unmatched_focal['_merge'] == 'left_only'].shape[0]

# Step 3: Display the results
print(f"Matched observations: {matched_observations}")
print(f"Observations in BenchmarkSummary but not in FocalSummary: {benchmark_unmatched_count}")
print(f"Observations in FocalSummary but not in BenchmarkSummary: {focal_unmatched_count}")


Matched observations: 51156
Observations in BenchmarkSummary but not in FocalSummary: 3591
Observations in FocalSummary but not in BenchmarkSummary: 23632


In [ ]:
# Step 1: Define the dummy variables and create the BlueState info DataFrame
state_info = {
    'region': ['AL', 'AR', 'AZ', 'CA', 'CO', 'FL', 'GA', 'IL', 'IN',
               'KS', 'KY', 'MA', 'MD', 'MI', 'MO', 'MS', 'NC', 'NH',
               'NJ', 'NY', 'OH', 'OK', 'PA', 'TN', 'TX', 'VA', 'WI', 'WV'],
    'BlueState': [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
                  0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
}

state_info_df = pd.DataFrame(state_info)

# Step 2: Merge BlueState information into MergedSummary
MergedSummary = MergedSummary.merge(state_info_df[['region', 'BlueState']], on='region', how='left')


# Now MergedSummary will have the 'BlueState' and 'Blue_Post' interaction term.
MergedSummary.head()


,region,date,dayofweek_x,Nunits_x,SumDailyVisits_x,MaxDailyVisitsPerUnit_x,MinDailyVisitsPerUnit_x,MedDailyVisitsPerUnit_x,dayofweek_y,Nunits_y,SumDailyVisits_y,MaxDailyVisitsPerUnit_y,MinDailyVisitsPerUnit_y,MedDailyVisitsPerUnit_y,BlueState
0,AL,2018-01-01,0,4,184,52,37,47.5,0,1,7,7,7,7.0,0
1,AL,2018-01-02,1,4,48,15,9,12.0,1,1,10,10,10,10.0,0
2,AL,2018-01-03,2,4,130,38,28,32.0,2,1,18,18,18,18.0,0
3,AL,2018-01-04,3,4,127,35,29,31.5,3,1,14,14,14,14.0,0
4,AL,2018-01-05,4,4,171,56,33,41.0,4,1,30,30,30,30.0,0


In [ ]:
# save merged summary
MergedSummary.to_csv('/content/drive/MyDrive/ECON_433_Project_3_data/MergedSummary.csv', index=False)

The above output shows the obsevations for each category asked for

Matched Parts: These represent regions and dates where both chains operate, so their data align.

Unmatched Parts: Differences can arise because:
Focal operates in more regions (42 vs. 30), so some Focal regions have no Benchmark counterparts.
Business schedules or observations might differ, leading to region-days present in one dataset but missing in the other.

Part 3: Group comparison in the merged data

Question 3a (2 point): Now using MergedSummary, you can compare the two chains.

Questions to answer:

Calculate average SumDailyVisits for the two chains.  Show the results in a table.
How does the average SumDailyVisits differ between the two chains? (Write at least one sentence about your takeaway from the table.)



In [ ]:
# Calculate the average SumDailyVisits for both chains
average_sum_daily_visits = pd.DataFrame({
    'Chain': ['Benchmark', 'Focal'],
    'AverageSumDailyVisits': [
        MergedSummary['SumDailyVisits_x'].mean(),
        MergedSummary['SumDailyVisits_y'].mean()
    ]
})

# Display the results in a table format
print(tabulate(average_sum_daily_visits, headers='keys', tablefmt='grid', showindex=False))

# Calculate the difference
chain_difference = average_sum_daily_visits['AverageSumDailyVisits'].diff().iloc[-1]
print(f"\nThe average SumDailyVisits for each chain differs by: {chain_difference:.2f}.")


+-----------+-------------------------+
| Chain     |   AverageSumDailyVisits |
+===========+=========================+
| Benchmark |                 167.11  |
+-----------+-------------------------+
| Focal     |                 191.509 |
+-----------+-------------------------+

The average SumDailyVisits for each chain differs by: 24.40.


From the table I can see that my Focal data has 24.4 more average sumdailyvisits.

Question 3b (2 points): Perform a statistical test on the difference of average SumDailyVisits between the two chains.

Questions to answer:

Is the difference statistically significant from zero with 95% confidence?
Does it confirm your prior in Question 3a?

In [ ]:
import scipy.stats as stats

# Extract the daily visit sums for both chains
benchmark_visits = MergedSummary['SumDailyVisits_x']
focal_visits = MergedSummary['SumDailyVisits_y']

# Perform the independent t-test
t_stat, p_value = stats.ttest_ind(benchmark_visits, focal_visits, equal_var=False)

# Calculate the means for both chains
benchmark_mean = benchmark_visits.mean()
focal_mean = focal_visits.mean()

# Display the t-test results
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Calculate the 95% confidence interval
confidence_level = 0.95
degrees_freedom = len(benchmark_visits) + len(focal_visits) - 2
confidence_interval = stats.t.interval(confidence_level, degrees_freedom, loc=benchmark_mean - focal_mean, scale=stats.sem(benchmark_visits - focal_visits))

print(f"95% Confidence Interval for the difference in means: {confidence_interval}")


T-statistic: -16.4582
P-value: 0.0000
95% Confidence Interval for the difference in means: (-26.512503739201087, -22.28587768233306)


See Output above

Yes, the difference is statistically significant from zero at the 95% confidence level. The p-value is 0.0000, which is much less than the typical alpha level of 0.05. This indicates that we can reject the null hypothesis, which states that there is no difference in the average SumDailyVisits between the two chains.
the results confirm the prior findings from Question 3a, where the average SumDailyVisits for the two chains differed by 24.40. The statistical significance of this difference suggests that the observed disparity in daily visits is not due to random chance, supporting the conclusion that Focal and Benchmark differ meaningfully in their performance in terms of daily visits.

Part 4: Basic regression analysis



Question 4a (1 points): Focus on the data MergedSummary. Define a dummy “Post03132020” equal to one if date is on or after March 13, 2020. Define a dummy "BlueState" equal to 1 if a state has a higher fraction of population voting for Hilary Clinton according to the 2016 Presidential Election outcome (see NY Times Links to an external site.for the color of each state). Define “Blue_Post” as BlueState times Post03132020. Regress ln(SumDailyVisits+1) of your own chain on BlueState, Post03132020, and Blue_Post.

Questions to answer:

Comment on the estimated coefficient for each of the three variables: Are they statistically significant from zero with 95% confidence? What is the economic meaning of each coefficient?
Do you accept or reject the null hypothesis “H0: The national emergency announced on March 13, 2020 does not make a difference on average SumDailyVisits of your own chain between red and blue states”? Explain how you arrive at this conclusion.




In [ ]:
import statsmodels.api as sm

# Step 1: Define the dummy variables
MergedSummary['Post03132020'] = (MergedSummary['date'] >= '2020-03-13').astype(int)

# Step 2: Create state_info DataFrame with BlueState information
state_info = {
    'region': ['AL', 'AR', 'AZ', 'CA', 'CO', 'FL', 'GA', 'IL', 'IN',
               'KS', 'KY', 'MA', 'MD', 'MI', 'MO', 'MS', 'NC', 'NH',
               'NJ', 'NY', 'OH', 'OK', 'PA', 'TN', 'TX', 'VA', 'WI', 'WV'],
    'BlueState': [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
                  0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
}

state_info_df = pd.DataFrame(state_info)

# Merge BlueState information into MergedSummary
MergedSummary = MergedSummary.merge(state_info_df, on='region', how='left')

# Create interaction term
MergedSummary['Blue_Post'] = MergedSummary['BlueState'] * MergedSummary['Post03132020']


In [ ]:
#Save MergedSummary to CSV
MergedSummary.to_csv('/content/drive/MyDrive/ECON_433_Project_3_data/MergedSummary.csv', index=False)
MergedSummary.head()

,region,date,Nunits_x,SumDailyVisits_x,MaxDailyVisitsPerUnit_x,MinDailyVisitsPerUnit_x,MedDailyVisitsPerUnit_x,Nunits_y,SumDailyVisits_y,MaxDailyVisitsPerUnit_y,MinDailyVisitsPerUnit_y,MedDailyVisitsPerUnit_y,Post03132020,BlueState,Blue_Post
0,AL,2018-01-01,4,184,52,37,47.5,1,7,7,7,7.0,0,0,0
1,AL,2018-01-02,4,48,15,9,12.0,1,10,10,10,10.0,0,0,0
2,AL,2018-01-03,4,130,38,28,32.0,1,18,18,18,18.0,0,0,0
3,AL,2018-01-04,4,127,35,29,31.5,1,14,14,14,14.0,0,0,0
4,AL,2018-01-05,4,171,56,33,41.0,1,30,30,30,30.0,0,0,0


In [ ]:
import statsmodels.api as sm

# Step 1: Define the dummy variables
MergedSummary['Post03132020'] = (MergedSummary['date'] >= '2020-03-13').astype(int)

# Step 2: Create state_info DataFrame with BlueState information
state_info = {
    'region': ['AL', 'AR', 'AZ', 'CA', 'CO', 'FL', 'GA', 'IL', 'IN',
               'KS', 'KY', 'MA', 'MD', 'MI', 'MO', 'MS', 'NC', 'NH',
               'NJ', 'NY', 'OH', 'OK', 'PA', 'TN', 'TX', 'VA', 'WI', 'WV'],
    'BlueState': [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
                  0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
}

state_info_df = pd.DataFrame(state_info)

# Merge BlueState information into MergedSummary
MergedSummary = MergedSummary.merge(state_info_df, on='region', how='left')

# Create interaction term
MergedSummary['Blue_Post'] = MergedSummary['BlueState'] * MergedSummary['Post03132020']

# Step 3: Perform the regression
X = MergedSummary[['BlueState', 'Post03132020', 'Blue_Post']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = np.log(MergedSummary['SumDailyVisits_x'] + 1)

model = sm.OLS(y, X).fit()

# Step 4: Print the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:       SumDailyVisits_x   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     3702.
Date:                Mon, 11 Nov 2024   Prob (F-statistic):               0.00
Time:                        16:26:59   Log-Likelihood:                -77196.
No. Observations:               51156   AIC:                         1.544e+05
Df Residuals:                   51152   BIC:                         1.544e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.6333      0.009    522.651   

Comment on the estimated coefficient for each of the three variables (accounting for log transformation with plus one):
a. Constant (Intercept):

Coefficient: 4.6333
Interpretation: The constant represents the baseline average number of visits in red states before March 13, 2020. Since the model uses log(SumDailyVisits + 1), the baseline is exp(4.6333) - 1 ≈ 102 visits on average, accounting for the plus one. This coefficient is statistically significant (p-value = 0.000).
b. BlueState:

Coefficient: -0.8186
Interpretation: In blue states, the average number of daily visits was approximately exp(-0.8186) - 1 ≈ -0.56 (or 56% lower) compared to red states before March 13, 2020, accounting for the plus one in the log transformation. The negative coefficient is statistically significant (p-value = 0.000).
c. Post03132020 (National Emergency Announcement):

Coefficient: 0.3027
Interpretation: After March 13, 2020, daily visits increased by exp(0.3027) - 1 ≈ 35.3% overall, regardless of whether the state was red or blue, reflecting the recovery in visits post-announcement. This coefficient is statistically significant (p-value = 0.000).
d. Blue_Post (Interaction between BlueState and Post03132020):

Coefficient: -0.4244
Interpretation: In blue states, post-March 13, 2020, the increase in daily visits was exp(-0.4244) - 1 ≈ -0.35 (or 34.6% lower) compared to red states. This indicates that the increase in visits after the national emergency was less pronounced in blue states than in red states. This coefficient is statistically significant (p-value = 0.000).

Do you accept or reject the null hypothesis?
Null Hypothesis: H0: The national emergency on March 13, 2020 does not make a difference in average SumDailyVisits between red and blue states.
Conclusion: We reject the null hypothesis. The significant interaction term (Blue_Post, p-value = 0.000) shows that the national emergency had a different impact on visits between red and blue states, with blue states seeing a smaller recovery or larger decline in visits compared to red states.

Economic Meaning of Each Coefficient (with the log transformation in mind):

Intercept (4.6333): Represents a baseline of approximately 102 visits in red states before the pandemic (accounting for the +1 in the log).
BlueState (-0.8186): Blue states had about 56% fewer visits than red states before March 13, 2020.
Post03132020 (0.3027): Visits increased by about 35.3% after the national emergency, across both red and blue states.
Blue_Post (-0.4244): Blue states saw a 34.6% smaller increase in visits post-March 13, 2020, compared to red states.







Question 4b (1 points):

Questions to answer:

How would your answer to Question 4a change if:
(1) you add state fixed effects?

(2) you add date fixed effects? and

(3) you add both state fixed effects and date fixed effects?

Which specification would you prefer for the hypothesis testing?



In [ ]:
pip install linearmodels

State Fixed effect below.

In [ ]:
from linearmodels.panel import PanelOLS

# Setting a MultiIndex for panel data
MergedSummary.set_index(['region', 'date'], inplace=True)

# Step 4: Define the dependent variable
y = np.log(MergedSummary['SumDailyVisits_x'] + 1)

# Step 5: Define the independent variables
X = MergedSummary[['BlueState', 'Post03132020', 'Blue_Post']]

# Step 6: Fit the fixed effects model
mod = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
results = mod.fit(cov_type='clustered')

# Step 7: Print the regression results
print(results)

<ipython-input-152-5aa8e7bc8145>:14: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

BlueState

  results = mod.fit(cov_type='clustered')


                          PanelOLS Estimation Summary                           
Dep. Variable:       SumDailyVisits_x   R-squared:                        0.0380
Estimator:                   PanelOLS   R-squared (Between):              0.0440
No. Observations:               51156   R-squared (Within):               0.0380
Date:                Mon, Nov 11 2024   R-squared (Overall):              0.0439
Time:                        16:27:07   Log-likelihood                -5.026e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1008.5
Entities:                          28   P-value                           0.0000
Avg Obs:                       1827.0   Distribution:                 F(2,51126)
Min Obs:                       1827.0                                           
Max Obs:                       1827.0   F-statistic (robust):             1075.7
                            

Both State and date Fixed effects below.

In [ ]:
# Define the dependent and independent variables
y = np.log(MergedSummary['SumDailyVisits_x'] + 1)  # Dependent variable
X = MergedSummary[['BlueState', 'Post03132020', 'Blue_Post']]  # Independent variables

# Running the regression with both entity and time effects
mod_both = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)
results_both = mod_both.fit(cov_type='clustered')

# Print the results
print("Results with Both Entity and Time Effects:")
print(results_both)

<ipython-input-153-353a113f2b28>:7: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

BlueState, Post03132020

  results_both = mod_both.fit(cov_type='clustered')


Results with Both Entity and Time Effects:
                          PanelOLS Estimation Summary                           
Dep. Variable:       SumDailyVisits_x   R-squared:                        0.0653
Estimator:                   PanelOLS   R-squared (Between):             -0.0282
No. Observations:               51156   R-squared (Within):              -0.0140
Date:                Mon, Nov 11 2024   R-squared (Overall):             -0.0280
Time:                        16:27:11   Log-likelihood                -2.203e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3442.0
Entities:                          28   P-value                           0.0000
Avg Obs:                       1827.0   Distribution:                 F(1,49301)
Min Obs:                       1827.0                                           
Max Obs:                       1827.0   F-statistic (robust):     


1. Adding State Fixed Effects:
State fixed effects account for any unchanging differences across states that could influence daily visits, such as state-specific policies, demographics, or consumer behavior patterns that remain consistent over time.

Impact: The coefficient for BlueState was fully absorbed in the model, indicating that it is perfectly collinear with the state fixed effects. This absorption means that the BlueState variable does not provide additional explanatory power for variations in daily visits, as its influence is already captured by the fixed effects.

Interpretation: The focus of the analysis shifts from identifying general differences between blue and red states to examining changes within each state over time. This approach allows for a more nuanced understanding of how visit patterns evolved in individual states rather than attributing differences solely to state characteristics.

2. Adding Date Fixed Effects:
Date fixed effects control for any factors that uniformly affect all states at different points in time, such as economic conditions, seasons, or nationwide policies.

Impact: The absorption of the Post03132020 variable suggests that it does not provide additional explanatory power beyond what is captured by the date fixed effects, which control for the nationwide impact of the national emergency announcement.

Interpretation: The model emphasizes state-specific changes around March 13, 2020, rather than broader nationwide trends, allowing for a clearer understanding of how each state's daily visits varied in response to the emergency.

3. Adding Both State and Date Fixed Effects:
Incorporating both fixed effects removes the influence of unobserved state characteristics (like demographic trends or local laws) and any common time-related shocks that impact all states simultaneously.

Impact: With both fixed effects included, the original coefficients for BlueState and Post03132020 were fully absorbed, indicating they do not contribute additional explanatory variance to the model. The interaction term Blue_Post, however, remains significant, capturing the differential impact of the national emergency on daily visits between blue and red states.

Interpretation: This specification allows for the cleanest examination of how the national emergency affected daily visits in red versus blue states, focusing solely on the variations observed within each state around that pivotal date.

Conclusion:
The model that includes both state and date fixed effects is preferred because it effectively removes the influence of unobserved state characteristics and common time shocks. This setup provides a  framework to isolate the effect of the national emergency on daily visits in red versus blue states, thereby reducing potential biases in your analysis.

Both Fixed Effects: Offer the most precise estimates by isolating within-state changes in response to the national emergency, providing a more accurate test of your hypothesis regarding how the emergency influenced visit patterns across different political landscapes.


Question 4c (2 points): Now suppose you replace the dependent variable as [ln(SumDailyVisits+1) of your own chain - ln(SumDailyVisits+1) of your benchmark chain].

Questions to answer:

How would your findings in Question 4a and Question 4b change?
What may explain these changes?

In [ ]:
# Step 1: Calculate the difference in logged daily visits
MergedSummary['DiffLogVisits'] = np.log(MergedSummary['SumDailyVisits_y'] + 1) - np.log(MergedSummary['SumDailyVisits_x'] + 1)

# Step 2: Create the model matrix
# Create interaction term
MergedSummary['Blue_Post'] = MergedSummary['BlueState'] * MergedSummary['Post03132020']

# Step 3: Perform the regression with the difference in logged visits as the dependent variable
X = MergedSummary[['BlueState', 'Post03132020', 'Blue_Post']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = MergedSummary['DiffLogVisits']

# Run Ordinary Least Squares regression
model = sm.OLS(y, X).fit()

# Print the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          DiffLogVisits   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     1235.
Date:                Mon, 11 Nov 2024   Prob (F-statistic):               0.00
Time:                        16:27:12   Log-Likelihood:                -81992.
No. Observations:               51156   AIC:                         1.640e+05
Df Residuals:                   51152   BIC:                         1.640e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.2283      0.010     23.443   

Changes to Question 4a
Constant (Intercept): Represents the baseline difference in logged daily visits between Chili's Grill and Bar and the benchmark chain (Cracker Barrel) before March 13, 2020, rather than absolute levels.

BlueState: Reflects the difference in visits between blue and red states for both chains, indicating if the impact of state affiliation affects both similarly.

Post03132020: Measures how the national emergency announcement changed the relative performance of Chili's compared to Cracker Barrel, rather than absolute visit changes.

Blue_Post: Indicates how the national emergency's impact on visits differed between blue and red states in terms of relative performance between the two chains.

Changes to Question 4b
State Fixed Effects: Controls for state-level variations, focusing on changes in relative performance rather than fixed state characteristics.

Date Fixed Effects: Accounts for time-related factors affecting both chains, isolating the effects of the national emergency on their relative performance.

Both Fixed Effects: Provides the most precise estimates by controlling for both state and time influences, clarifying how external events affect the relative performance of the chains.

Explanation for Changes
Focus on Relative Performance: The new dependent variable highlights how external factors impact the competitive dynamics between Chili's and Cracker Barrel rather than absolute performance levels.

Controlled Variance: Fixed effects reduce confounding factors, allowing for a clearer analysis of how specific events influence the relative differences in visits.

In summary, this new approach shifts the focus from individual performance to competitive dynamics, leading to different interpretations and insights regarding the effects of external events on the chains.

In [ ]:
#Save MergedSummary to CSV
MergedSummary.to_csv('/content/drive/MyDrive/ECON_433_Project_3_data/MergedSummary.csv', index=False)
MergedSummary.head()

Nunits_x  SumDailyVisits_x  MaxDailyVisitsPerUnit_x  \
region date                                                              
AL     2018-01-01         4               184                       52   
       2018-01-02         4                48                       15   
       2018-01-03         4               130                       38   
       2018-01-04         4               127                       35   
       2018-01-05         4               171                       56   

                   MinDailyVisitsPerUnit_x  MedDailyVisitsPerUnit_x  Nunits_y  \
region date                                                                     
AL     2018-01-01                       37                     47.5         1   
       2018-01-02                        9                     12.0         1   
       2018-01-03                       28                     32.0         1   
       2018-01-04                       29                     31.5         1   
       2018-01-05                       33                     41.0         1   

                   SumDailyVisits_y  MaxDailyVisitsPerUnit_y  \
region date                                                    
AL     2018-01-01                 7                        7   
       2018-01-02                10                       10   
       2018-01-03                18                       18   
       2018-01-04                14                       14   
       2018-01-05                30                       30   

                   MinDailyVisitsPerUnit_y  MedDailyVisitsPerUnit_y  \
region date                                                           
AL     2018-01-01                        7                      7.0   
       2018-01-02                       10                     10.0   
       2018-01-03                       18                     18.0   
       2018-01-04                       14                     14.0   
       2018-01-05                       30                     30.0   

                   Post03132020  BlueState_x  Blue_Post  BlueState_y  \
region date                                                            
AL     2018-01-01             0            0          0            0   
       2018-01-02             0            0          0            0   
       2018-01-03             0            0          0            0   
       2018-01-04             0            0          0            0   
       2018-01-05             0            0          0            0   

                   BlueState  DiffLogVisits  
region date                                  
AL     2018-01-01          0      -3.140914  
       2018-01-02          0      -1.493925  
       2018-01-03          0      -1.930758  
       2018-01-04          0      -2.143980  
       2018-01-05          0      -1.713507